# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["KERAS_BACKEND"] = "torch"

import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Rescaling, Dropout, Resizing
from keras.layers import RandomFlip, RandomTranslation, RandomRotation, RandomZoom, MultiHeadAttention, AugMix, RandAugment
from keras.models import Sequential
from keras.metrics import F1Score, Precision, AUC
from keras.losses import CategoricalFocalCrossentropy, BinaryFocalCrossentropy
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.optimizers.schedules import ExponentialDecay, CosineDecay
from keras.optimizers import AdamW

from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, ADASYN, KMeansSMOTE
from imblearn.under_sampling import TomekLinks, RepeatedEditedNearestNeighbours
from imblearn.combine import SMOTETomek, SMOTEENN
from sklearn.utils import compute_class_weight

from PIL import Image

# Move images into class folders

In [ ]:
def move_and_separate_images(list_of_folders, df_metadata):
    df_metadata = df_metadata.set_index('image_id', drop=True)
    count = 0
    for folder in list_of_folders:
        for root, dirs, files in os.walk(folder):
            for file in files:
                try:
                    os.makedirs(f"dataset/{df_metadata.loc[file.strip('.jpg'), 'dx']}")
                except:
                    source=os.path.join(root, file)
                    destination=os.path.join('dataset', df_metadata.loc[file.strip('.jpg'), 'dx'], file)
                    os.rename(source,destination)
                    count += 1
                    if count%100 == 0:
                        print(f'{count} images were processed')

In [ ]:
df = pd.read_csv('HAM10000_metadata.csv')

move_and_separate_images(['HAM10000_images_part_1', 'HAM10000_images_part_2'], df)

# Move images into cancerous/non cancerous folders

In [ ]:
class_mapping = {
    "akiec": "canc",
    "bcc": "canc",
    "bkl": "nocanc",
    "df": "nocanc",
    "mel": "canc",
    "nv": "nocanc",
    "vasc": "nocanc"
}

In [ ]:
def move_and_separate_images(list_of_folders, df_metadata):
    df_metadata = df_metadata.set_index('image_id', drop=True)
    count = 0
    for folder in list_of_folders:
        for root, dirs, files in os.walk(folder):
            for file in files:
                try:
                    os.makedirs(f"dataset/{class_mapping[df_metadata.loc[file.strip('.jpg'), 'dx']]}")
                except:
                    source=os.path.join(root, file)
                    destination=os.path.join('dataset', class_mapping[df_metadata.loc[file.strip('.jpg'), 'dx']], file)
                    os.rename(source,destination)
                    count += 1
                    if count%100 == 0:
                        print(f'{count} images were processed')

In [ ]:
df = pd.read_csv('HAM10000_metadata.csv')

move_and_separate_images(['HAM10000_images_part_1', 'HAM10000_images_part_2'], df)

# Make arrays

In [ ]:
def array_from_images(folder, df_metadata, dict_of_labels, h=224, w=224, channels=3):
    # Create an array of images and labels the size of the number of pictures
    nb_files = 0
    for root, dirs, files in os.walk(folder):
        for file in files:
            nb_files += 1
    array = np.zeros(shape=(nb_files, h, w, channels))
    labels = np.zeros(shape=(nb_files,))

    # Check the name and fill array and labels
    df_metadata = df_metadata.set_index('image_id', drop=True)
    count = 0

    for root, dirs, files in os.walk(folder):
        for file in files:
            with Image.open(os.path.join(root, file)) as im:
                array[count,:,:,:] = np.asarray(im.resize((h,w)))
                try:
                    labels[count] = dict_of_labels[class_mapping[df_metadata.loc[file.strip('.jpg'), 'dx']]]
                except:
                    labels[count] = dict_of_labels[df_metadata.loc[file.strip('.jpg'), 'dx']]
                count += 1
                if count%100 == 0:
                    print(f'{count} images were processed')
    return array, labels

In [ ]:
class_mapping = {
    "akiec": "canc",
    "bcc": "canc",
    "bkl": "nocanc",
    "df": "nocanc",
    "mel": "canc",
    "nv": "nocanc",
    "vasc": "nocanc"
}

In [ ]:
df = pd.read_csv('HAM10000_metadata.csv')
# dict_label = {k:v for k,v in zip(['nocanc', 'canc'], range(2))}
dict_label = {k:v for k,v in zip(list(class_mapping.keys()), range(7))}
size = 224

X, y = array_from_images('dataset/', df, dict_label, h=size, w=size)


In [ ]:
np.savez_compressed(f'X-{size}x{size}', X, allow_pickle=True)
np.savez_compressed(f'y-{size}x{size}', y, allow_pickle=True)

# Load arrays

In [2]:
size = 224
X = np.load(f'X-{size}x{size}.npz')['arr_0']
y = np.load(f'y-{size}x{size}.npz')['arr_0']

# Compute class weights

In [3]:
class_weights = {i:(len(y)/(7 * sum(y==i))).item() for i in range(7)}

In [4]:
# class_weights = compute_class_weight(class_weight="balanced", classes=np.array(range(7)), y=y)

# Split arrays

In [5]:
from sklearn.model_selection import train_test_split

X_train, X, y_train, y = train_test_split(X, y, stratify=y, test_size=0.2, random_state=38)



X_test, X_val, y_test, y_val = train_test_split(X, y, stratify=y, test_size=0.5, random_state=38)

del X,y

# SMOTETOMEK

In [ ]:
shape_origin = X_train.shape

X_train = np.reshape(X_train, (shape_origin[0], shape_origin[1]*shape_origin[2]*shape_origin[3]))

smotetomek = SMOTETomek(random_state=38, n_jobs=-1)

X_train, y_train = smotetomek.fit_resample(X_train, y_train)

X_train = np.reshape(X_train, (X_train.shape[0], shape_origin[1], shape_origin[2], shape_origin[3]))



# One Hot Encoding of labels

In [6]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

# Helper functions

In [7]:
def get_earlystopping(patience=10):
    early_stopping = EarlyStopping(
    monitor='val_mean_average_precision',
    mode='max',
    patience=patience,
    verbose=1,
    restore_best_weights=True)
    return early_stopping

In [8]:
def plot_learning_curves(model):
    fig, ax = plt.subplots(1,3, figsize=(15,5))
    ax[0].plot(model.history.history['val_f1_score'], label='val_f1_score')
    ax[0].plot(model.history.history['f1_score'], label='f1_score')
    ax[0].legend()
    try:
        ax[1].plot(model.history.history['val_average_precision'], label='val_avg_prec')
        ax[1].plot(model.history.history['average_precision'], label='avg_prec')
        ax[1].legend()
    except:
        pass
    #     ax[1].plot(model.history.history['val_precision_1'], label='val_precision_1')
    #     ax[1].plot(model.history.history['precision_1'], label='precision_1')
    #     ax[1].legend()
    ax[2].plot(model.history.history['val_loss'], label='val_loss')
    ax[2].plot(model.history.history['loss'], label='loss')
    ax[2].legend()
    fig.show;

In [9]:
def get_analysis(model, testX, testy):
    plot_learning_curves(model)
    loss, f1, avg_prec = model.evaluate(testX, testy)
    print(f'The model gave')
    print(f'Loss: {loss:.2f}')
    print(f'F1 Macro: {f1:.2f}')
    print(f'Avg Prec: {avg_prec:.2f}')
    y_pred = model.predict(testX)
    y_res = np.argmax(y_pred, axis=1)
    print(classification_report(testy, y_res))
    return y_pred, y_res

In [10]:
def get_analysis_cat(model, testX, testy):
    plot_learning_curves(model)
    loss, f1, avg_prec = model.evaluate(testX, testy)
    print(f'The model gave')
    print(f'Loss: {loss:.2f}')
    print(f'F1 Macro: {f1:.2f}')
    print(f'Avg Prec: {avg_prec:.2f}')
    predy = model.predict(testX)
    resy = to_categorical(np.argmax(predy, axis=1))
    print(classification_report(testy,resy))
    return predy, resy

In [11]:
def compile_and_train(model, loss, opt, metrics, epochs, patience=None, steps=None, class_weight=None):
    model.compile(loss=loss,
                optimizer=opt,
                metrics=metrics)

    model.summary()

    if patience != None:
        model.fit(
            X_train,
            y_train,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=[get_earlystopping(patience)],
            steps_per_epoch=steps,
            class_weight=class_weight
            )
    else:
        model.fit(
            X_train,
            y_train,
            epochs=epochs,
            validation_data=(X_val, y_val),
            steps_per_epoch=steps,
            class_weight=class_weight
            )

    return model

# Model

In [15]:
from keras.applications import EfficientNetV2B0, EfficientNetV2S

effnet = EfficientNetV2B0(
    include_top=True,
    input_shape=X_train.shape[1:],
    weights=None,
    classes=7,
    classifier_activation="softmax",
)

model = Sequential([
    Input(shape=(X_train.shape[1:])),
    Rescaling(scale=1./255),
    RandAugment(value_range=(0,1), num_ops=1, factor=0.1),
    effnet, 
    # Dropout(0.2),
    # Flatten(),
    # Dense(16, activation='relu'),
    # Dense(7, activation='softmax')
])

In [16]:
def mean_average_precision(y_true, y_pred):
    aps = []
    for c in range(7):
        ap = average_precision_score(y_true.detach().cpu().numpy()[:, c], y_pred.detach().cpu().numpy()[:, c])
        aps.append(ap)

    mAP = sum(aps) / len(aps)
    return mAP


In [17]:
lr_schedule = CosineDecay(initial_learning_rate=1e-3, decay_steps=500, alpha=0.01) 
optimizer = AdamW(learning_rate=lr_schedule, weight_decay=0.01)


history020 = compile_and_train(model,
                               loss=CategoricalFocalCrossentropy(),
                               opt=optimizer,
                               metrics=[mean_average_precision, F1Score(average='macro')],
                               epochs=500,
                               patience=50, 
                            #    class_weight=class_weights
                )

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_3 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rand_augment_1 (RandAugment)    │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 7)              │     5,928,279 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,928,279 (22.61 MB)

 Trainable params: 5,867,671 (22.38 MB)

 Non-trainable params: 60,608 (236.75 KB)

Epoch 1/500
 24/251 ━━━━━━━━━━━━━━━━━━━━ 4:04 1s/step - f1_score: 0.1254 - loss: 0.5339 - mean_average_precision: 0.1910

KeyboardInterrupt: 

In [ ]:
pred, res = get_analysis_cat(model, X_test, y_test)

In [ ]:
torch.save(model, "model_avgprec_90_f1_69_env2b0_128px.pth")

In [ ]:
np.mean(list(class_weights.values()))